<a href="https://colab.research.google.com/github/camoragaq/C-uoh/blob/main/Clase8_threads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Threads en C

1. En este ejemplo repasaremos las principales funciones para el manejo de threads en C

In [1]:
%%writefile hilo.c

#include <stdio.h>
#include <string.h>
#include <pthread.h>

// Definimos una varibale global
int i = 2;

//funcion de inicio de thread
void* foo(void* p){
  printf("valor p = %i\n", * (int*)p);

  // función de salida
  pthread_exit(&i);
}

int main(void){
  // Declaramos el objeto tread y su identificador
  pthread_t id;

  int j = 1;
  //funcion para iniciar el hilo
  pthread_create(&id, NULL, foo, &j);
    
  int* ptr;

  // funcion que espera a foo y devuelve el valor en ptr
  pthread_join(id, (void**)&ptr);
  printf("Valor recibido desde el hijo: ");
  printf("%i\n", *ptr);
}

Writing hilo.c


In [10]:
!gcc hilo.c -o hilo -lpthread

In [11]:
!./hilo

valor p = 1
Valor recibido desde el hijo: 2


2. Otro ejemplo usando varios hilos

In [16]:
%%writefile hilos2.c

#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <pthread.h>
#include <errno.h>
#include <string.h>



// declaramos una variable global para ser usada en los hilos
int g = 0;

// funcion ejecutada por los hilos
void *mihilofun(void *vargp)
{
	// guardamos el id del hilo
	int *myid = (int *)vargp;

	// inicializamos una variable local
	 int s = 0;

	// cambiamos la variable local y global
  //	++s; ++g;
  int iter=rand() % 50;
  
  //incrementamos la variable global en inter 
  for(int j=0; j< iter; j++) ++s, ++g;
	// imprimimos ambas variables
	printf("ID Hilo: %d, Local: %d, Global: %d Random:%d\n", abs(*myid), s, g,iter);
}

//crea 5 hilos de manera dimamica y ejecuta la funcion mihilofun
int main()
{
  int numero_hilos=5;
  pthread_t *tid=(pthread_t *)malloc(numero_hilos * sizeof(pthread_t));
	// creamos 5 hilos
	for (int i = 0; i < numero_hilos; i++){
		//pthread_create(&tid, NULL, mihilofun, (void *)&tid);
    int err = pthread_create(&tid[i], NULL, &mihilofun, (void *)&(tid[i]));
    if (err != 0){
        printf("\ncan't create thread :[%s]", strerror(err));
    }
  }
  //esperamos que los hilos terminen
  for(int i=0; i< numero_hilos; ++i) pthread_join(tid[i], NULL);

printf("come back to main\n");
return 0;
}

Overwriting hilos2.c


In [17]:
!gcc hilos2.c -o hilos2 -lpthread

hilos2.c: In function ‘mihilofun’:
hilos2.c:27:19: error: ‘iter’ undeclared (first use in this function)
   for(int j=0; j< iter; j++) ++s, ++g;
                   ^~~~
hilos2.c:27:19: note: each undeclared identifier is reported only once for each function it appears in


In [9]:
!./hilos2

ID Hilo: 810162432, Local: 33, Global: 33 Random:33
ID Hilo: 826947840, Local: 36, Global: 69 Random:36
ID Hilo: 835340544, Local: 15, Global: 111 Random:15
ID Hilo: 843733248, Local: 27, Global: 96 Random:27
ID Hilo: 818555136, Local: 43, Global: 154 Random:43
come back to main


Ejercicio propuesto: Calcule el factorial usando 2 threads: